In [1]:
import json
from collections import namedtuple
import re

import pandas as pd

Plan = namedtuple("Plan", "context actions")

goldpath_file = "../data/goldsequences-0-1-2-3-4-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29.json"
#goldpath_file = "../data/goldsequences-0.json"

with open(goldpath_file) as file:
    data = json.load(file)


open/close OBJ - open/close a container
de/activate OBJ - activate/deactivate a device
connect OBJ to OBJ - connect electrical components
disconnect OBJ - disconnect electrical components
use OBJ [on OBJ] - use a device/item
look around describe - the current room
look at OBJ - describe an object in detail
look in OBJ - describe a container’s contents
read OBJ - read a note or book
move OBJ to OBJ - move an object to a container
pick up OBJ - move an object to the inventory
put down OBJ - drop an inventory item
pour OBJ into OBJ - pour a liquid into a container
dunk OBJ into OBJ - dunk a container into a liquid
mix OBJ - chemically mix a container
go to LOC - move to a new location
eat OBJ - eat a food
flush OBJ - flush a toilet
focus on OBJ - signal intent on a task object

In [2]:
all_plans = {}
variation_list = []
task_id_list = []
for task_id in list(data.keys()):
    for variation in range(0, len(data[task_id]['goldActionSequences'])):
        variation_list.append(variation)
        trajectory = data[task_id]['goldActionSequences'][variation]
        task = trajectory['taskDescription']
        goal = task.split('.')[0]
        if goal not in all_plans:
            all_plans[goal] = []
            task_id_list.append(data[task_id]['taskName'])

        plan = []
        for step in trajectory['path']:
            action = step['action']
            if action != 'look around':
                plan.append(action)
            if step['isCompleted'] == 'true':
                break

        all_plans[goal].append({'plan': plan, 'variation': variation})

print("ok")


ok


In [3]:
data[task_id]['taskName']

'task-9-mendellian-genetics-(unknown-plant)'

In [4]:
#landmark: actions that are performed in every variation
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)

    # check length
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    else:
        return("no common elements")

common_plans = {}
for goal in all_plans.keys():
    plans = all_plans[goal]
    common_actions = []

    plan_a = plans[0]['plan']

    for plan in plans[1:]:
        plan = plan['plan']
        common_actions = common_member(plan_a, plan)
        plan_a = common_actions

    common_plans[goal] = common_actions


In [5]:
rows = []

plan_id = 0
for i, ((goal, plans), task_id) in enumerate(zip(all_plans.items(), task_id_list)):
    landmarks = common_plans[goal]
    for plan in plans:
        variation = plan['variation']
        for k, action in enumerate(plan['plan']):
            flag = True if action in landmarks else False
            rows.append({
                'action': action,
                'landmark': flag,
                'variation': variation,
                'order': k,
                'goal': goal,
                'task_id': task_id
            })
        plan_id += 1

df = pd.DataFrame(rows).sort_values(['task_id','variation', 'order'])

In [6]:
import os

for i, ((goal, plans), task_id) in enumerate(zip(all_plans.items(), task_id_list)):

    filename = goal.replace(" ", "_")
    #filepath = f"plans_csv/{task_id}/{filename.lower()}"
    filepath = f"../data/plans_csv/{task_id}/{filename.lower()}"
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    plans_df = df[df['goal'] == goal].sort_values(['variation', 'order'])
    #plans_df.to_csv(f"{filepath}/{filename}.csv", index=False)
    for variation in plans_df['variation'].unique():
        plans_df[plans_df['variation'] == variation].to_csv(f"{filepath}/{variation}.csv", index=False)

In [7]:
count_steps_df = df.groupby(["task_id", "goal", "variation"])['order'].count().reset_index()
count_steps_df = count_steps_df.groupby(["task_id", "goal"])['order'].mean().reset_index()
count_steps_df = count_steps_df.groupby(["task_id"])['order'].mean().reset_index()
count_steps_df.sort_values("order", ascending=False)

,task_id,order
25,task-8-inclined-plane-friction-(named-surfaces),153.437590
26,task-8-inclined-plane-friction-(unnamed-surfaces),131.697531
28,task-9-mendellian-genetics-(unknown-plant),119.754167
27,task-9-mendellian-genetics-(known-plant),119.355556
24,task-8-inclined-plane-determine-angle,97.369048
14,task-4-grow-fruit,66.476190
0,task-1-boil,64.571429
2,task-1-freeze,56.607143
3,task-1-melt,46.357143
1,task-1-change-the-state-of-matter-of,42.857143


In [ ]:
"""
! if your task is to boil an water AND you are not in the kitchen
    then
! go to the kitchen (you should see a door to the kitchen)
    + open door to the kitchen
    + go to the kitchen
! pick up metal pot (you should are in the kitchen)
    + open cupboard
    + pick up metal pot
+ pick up thermometer
! fill metal pot with water (must have metal pot in your inventory and you should see a sink)
    + pour metal pot into sink
    + move metal pot to sink
    + activate sink
    + deactivate sink
    + pick up metal pot
! boil water (You should have metal pot in your inventory with water and a thermometer, and you should see a stove)
    + deactivate stove
    + move metal pot to stove
    + activate stove
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
"""